In [ ]:
import numpy as np
import sympy
import math
import matplotlib.pyplot as plt
from pyodesys.symbolic import SymbolicSys
sympy.init_printing()
%matplotlib inline

In [ ]:
# 2 HNO2 -> H2O + NO + NO2; MassAction(EyringHS.fk('dH1', 'dS1'))
# 2 NO2 -> N2O4; MassAction(EyringHS.fk('dH2', 'dS2'))
#
# HNO2 H2O NO NO2 N2O4
def get_odesys(scaling=1):
    def rhs(t, y, p, backend=math):
        HNO2, H2O, NO, NO2, N2O4 = y
        dH1, dS1, dH2, dS2 = p
        R = 8.314
        T = 300 + 10*backend.sin(0.2*math.pi*t - math.pi/2)
        kB_h = 2.08366e10
        k1 = kB_h*T*backend.exp(dS1/R - dH1/(R*T))/scaling  # bimolecular => scaling**-1
        k2 = kB_h*T*backend.exp(dS2/R - dH2/(R*T))/scaling  # bimolecular => scaling**-1
        r1 = k1*HNO2**2
        r2 = k2*NO2**2
        return [-2*r1, r1, r1, r1 - 2*r2, r2]
    
    return SymbolicSys.from_callback(rhs, 5, 4, names='HNO2 H2O NO NO2 N2O4'.split(), param_names='dH1 dS1 dH2 dS2'.split())

In [ ]:
def integrate_and_plot(system, axes=None, scaling=1):
    init_y = [1*scaling, 55*scaling, 0, 0, 0]
    p = [85e3, 10, 70e3, 20]
    result = system.integrate(np.linspace(0, 60, 200), init_y, p, integrator='cvode', nsteps=5000)
    if axes is None:
        fig, axes = plt.subplots(1, 2, figsize=(10, 4))
    result.plot(ax=axes[0], indices=[0,2,4])
    result.plot(ax=axes[1], indices=[3])
    print({k: v for k, v in sorted(result.info.items()) if not k.startswith('internal')})
    return result

In [ ]:
def compare_autonomous(scaling):
    fig, all_axes = plt.subplots(3, 2, figsize=(10, 14))
    odesys = get_odesys(scaling)
    autsys = odesys.as_autonomous()
    res1 = integrate_and_plot(odesys, axes=all_axes[0, :], scaling=scaling)
    res2 = integrate_and_plot(autsys, axes=all_axes[1, :], scaling=scaling)
    assert np.allclose(res1.yout, res2.yout, atol=1e-6)
    diff = res1.yout-res2.yout
    res1.plot(ax=all_axes[2, 0], indices=[0,2,4], y=diff/scaling)
    res1.plot(ax=all_axes[2, 1], indices=[3], y=diff/scaling)

In [ ]:
compare_autonomous(1)

In [ ]:
compare_autonomous(1000)